In [1]:
from tg.grammar_ru.ml.corpus import CorpusReader
from tg.grammar_ru.common import Loc
from pathlib import Path

base_path = Loc.corpus_path/'example.base.zip'
reader = CorpusReader(Path(base_path))

In [2]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

alg = AntecedentCandidatesAlgorithm(reader.get_bundles(), 7)

c:\users\alexandra\desktop\grammar_ru\venv\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  torch.from_numpy(navec.pq.indexes),


Конечно, никто никогда в мире не писал замену имён собственных, поэтому напишем:

In [3]:
alg.pronoun_replacer

{'академия': 'женщина',
 'алевтина': 'женщина',
 'амелия': 'женщина',
 'андрий': 'мужчина',
 'аннегрета': 'женщина',
 'бэнд': 'мужчина',
 'ванко': 'мужчина',
 'верин': 'мужчина',
 'викторович': 'женщина',
 'глас': 'мужчина',
 'густав': 'мужчина',
 'денежкин': 'мужчина',
 'исповедание': 'существо',
 'кареолан': 'мужчина',
 'карл': 'мужчина',
 'каролина': 'женщина',
 'каспер': 'мужчина',
 'кик': 'мужчина',
 'кит': 'мужчина',
 'лилиана': 'женщина',
 'лилия': 'женщина',
 'марек': 'мужчина',
 'мартин': 'мужчина',
 'матвей': 'мужчина',
 'михал': 'мужчина',
 'никита': 'мужчина',
 'оливер': 'мужчина',
 'ополе': 'существо',
 'патрик': 'мужчина',
 'росинтеллектсистема': 'женщина',
 'снегур': 'мужчина',
 'стефанос': 'мужчина',
 'творец': 'мужчина',
 'тор': 'мужчина',
 'торам': 'мужчина',
 'франтишек': 'мужчина',
 'фредерик': 'мужчина',
 'ханна': 'женщина',
 'ханне': 'женщина',
 'чисовица': 'женщина',
 'ярина': 'женщина',
 'ярином': 'мужчина'}

In [2]:
db = reader.get_bundles().first()               

In [5]:
# то, что, вероятно, было выделением имён собственных в slovnet:
from tg.grammar_ru.ml.features import SlovnetFeaturizer

featurizer = SlovnetFeaturizer()
featurizer.featurize(db)
db.src[db.slovnet.POS == 'PROPN'].word.head(15)

0              Лилия
63          Закричав
65             Лилия
118           Голова
142            Лилия
152         Выманили
208            Лилия
220    Прикоснувшись
275            Лилия
333        Чисовицах
346        Девятерых
355           Лилией
370         Академии
372         Каролина
402            Лилия
Name: word, dtype: object

In [6]:
from tg.grammar_ru.ml.features import PyMorphyFeaturizer

featurizer = PyMorphyFeaturizer()
featurizer.featurize(db)
db.pymorphy['normal_form'] = db.pymorphy['normal_form'].apply(lambda x: alg.pronoun_replacer[x] if x in alg.pronoun_replacer else x)
db.pymorphy.head()

,normal_form,alternatives,score,delta_score,POS,animacy,gender,number,case,aspect,transitivity,person,tense,mood,voice,involvement
word_id,,,,,,,,,,,,,,,,
0,женщина,2,0.666666,0.333333,NOUN,anim,femn,sing,nomn,NaN,NaN,NaN,NaN,NaN,NaN,None
1,очнуться,1,1.000000,1.000000,VERB,NaN,femn,sing,NaN,perf,intr,NaN,past,indc,NaN,None
2,от,1,1.000000,1.000000,PREP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,суетливый,3,0.333333,0.000000,ADJF,NaN,masc,sing,gent,NaN,NaN,NaN,NaN,NaN,NaN,None
4,",",1,1.000000,1.000000,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [7]:
candidates_df = alg.get_candidates(db)
with_parents_df = alg.get_pronoun_parent(db, candidates_df)
with_parents_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id
0,10,0,0,9
1,18,0,1,19
2,18,10,0,19
3,30,0,2,31
4,30,10,1,31
...,...,...,...,...
303,921,907,4,922
304,921,914,3,922
305,921,917,2,922
306,921,918,1,922


In [8]:
from tg.grammar_ru.ml.features import NavecFeaturizer

product_df = with_parents_df.merge(db.pymorphy.normal_form.rename('pronoun_parent_norm'), left_on='pronoun_parent_id', right_index=True, how='left')
product_df = product_df.merge(db.pymorphy.normal_form.rename('candidate_word_norm'), left_on='candidate_word_id', right_index=True, how='left')
featurizer = NavecFeaturizer()
product_df['product'] = featurizer.get_glove_prod(product_df, 'pronoun_parent_norm', 'candidate_word_norm')
product_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,product
0,10,0,0,9,преследовать,женщина,2.660135
1,18,0,1,19,гнаться,женщина,-1.817963
2,18,10,0,19,гнаться,она,-0.213960
3,30,0,2,31,оставаться,женщина,7.911864
4,30,10,1,31,оставаться,она,8.539292
...,...,...,...,...,...,...,...
303,921,907,4,922,кубарем,жертва,-0.980053
304,921,914,3,922,кубарем,судьба,-1.951545
305,921,917,2,922,кубарем,она,-0.423255
306,921,918,1,922,кубарем,роль,-1.972080


In [9]:
# Можно ли улучшить выбор кандидатов, если отбрасывать детей родителя местоимения:
pronoun_parent_ids = db.slovnet[db.slovnet.index.isin(candidates_df['pronoun_word_id'])]['syntax_parent_id']
candidate_parent_ids = db.slovnet[db.slovnet.index.isin(candidates_df['candidate_word_id'])]['syntax_parent_id']
parents_compare = (candidates_df
 .merge(pronoun_parent_ids.to_frame().reset_index(), left_on='pronoun_word_id', right_on='word_id')
 .merge(candidate_parent_ids.to_frame().reset_index(), left_on='candidate_word_id', right_on='word_id'))
parents_compare[parents_compare['syntax_parent_id_x'] == parents_compare['syntax_parent_id_y']]

,pronoun_word_id,candidate_word_id,candidate_distance,word_id_x,syntax_parent_id_x,word_id_y,syntax_parent_id_y
303,917,914,0,917,916,914,916


In [10]:
product_df[product_df['pronoun_word_id'] == 917]

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,product
294,917,878,6,916,уготовить,паника,-2.770621
295,917,884,5,916,уготовить,женщина,-5.830828
296,917,894,4,916,уготовить,лестница,-2.799802
297,917,896,3,916,уготовить,женщина,-5.830828
298,917,905,2,916,уготовить,она,-5.512835
299,917,907,1,916,уготовить,жертва,-0.058948
300,917,914,0,916,уготовить,судьба,3.859842


На первый DataBundle всего один случай ошибки при совпадающем у местоимения и кандидата в антецеденты родителе, осталось бы ошибкой, вероятно, пока можно не трогать.

In [11]:
def get_big_deviation_only(df):
    product_groups = df.drop_duplicates(['pronoun_parent_norm','candidate_word_norm'], keep='last').groupby(['pronoun_word_id'])['product']
    maxes = product_groups.max()
    means = product_groups.mean()
    stds = product_groups.std()
    max_indices = product_groups.idxmax().dropna().astype(int)
    best_in_group = df.loc[max_indices]
    temp = ((product_groups.count() < 4) | (stds < maxes - means)).to_frame()
    return best_in_group[best_in_group.pronoun_word_id.isin(temp[temp['product']].index)]

get_big_deviation_only(product_df)

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent_norm,candidate_word_norm,product
0,10,0,0,9,преследовать,женщина,2.660135
2,18,10,0,19,гнаться,она,-0.213960
5,30,18,0,31,оставаться,она,8.539292
15,53,30,0,52,подвести,она,1.963640
20,56,53,0,57,упасть,она,7.820927
23,83,56,4,84,увидеть,она,10.808409
30,113,95,4,114,напугать,девушка,5.793210
40,122,113,1,123,чувствовать,она,10.140821
42,145,105,6,146,подкараулить,кирпичный,-0.722384
53,160,124,2,163,успеть,тревога,5.222582


Брать местоимения в кандидаты было плохой идеей, далее без них.

In [2]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

alg = AntecedentCandidatesAlgorithm(reader.get_bundles(), 7)

c:\users\alexandra\desktop\grammar_ru\venv\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  torch.from_numpy(navec.pq.indexes),


In [4]:
alg.run_full(db).drop(['pronoun_parent_id'], axis=1)

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_norm,candidate_word_norm,product
0,10,0,0,преследовать,женщина,2.660135
1,18,0,0,гнаться,женщина,-1.817963
2,30,0,0,оставаться,женщина,7.911864
9,53,0,0,подвести,женщина,-0.543144
10,56,0,0,упасть,женщина,5.786184
13,83,65,2,увидеть,женщина,8.552348
18,113,95,4,напугать,девушка,5.793210
31,145,104,5,подкараулить,глухой,-0.551586
42,160,124,1,успеть,тревога,5.222582
50,174,142,0,показаться,женщина,7.261024


In [76]:
from tg.grammar_ru.ml.features import NavecFeaturizer
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def inflect(word, case):
    return morph.parse(word)[0].inflect({'sing', case}).word

def get_inflected_candidates(db, with_parents_df):
    inflect_base = with_parents_df.merge(db.pymorphy, left_on='pronoun_word_id', right_on='word_id')[['candidate_word_id', 
                                                                                                 'case']]
    inflect_base = inflect_base.merge(db.pymorphy.reset_index()[['word_id', 'normal_form']], left_on='candidate_word_id', right_on='word_id', how='left')[['case', 'normal_form']]
    return inflect_base.apply(lambda x: inflect(x['normal_form'], x['case']), axis=1)

candidates_df = alg.get_candidates(db)
with_parents_df = alg.get_pronoun_parent(db, candidates_df)
product_df = with_parents_df.merge(db.src.word.str.lower().rename('pronoun_parent'), left_on='pronoun_parent_id', right_index=True, how='left')
product_df['inflected_candidate'] = get_inflected_candidates(db, with_parents_df)
featurizer = NavecFeaturizer()
product_df['product'] = featurizer.get_glove_prod(product_df, 'pronoun_parent', 'inflected_candidate')
product_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent,inflected_candidate,product
0,10,0,0,9,преследовал,женщину,8.436619
1,18,0,0,19,гналась,женщина,1.514307
2,30,0,0,31,оставался,женщине,3.133473
3,48,3,5,46,оторваться,суетливого,-0.075657
4,48,5,4,46,оторваться,путаного,-2.126049
...,...,...,...,...,...,...,...
291,921,896,4,922,кубарем,женщина,-0.210612
292,921,907,3,922,кубарем,жертва,-0.980053
293,921,914,2,922,кубарем,судьба,-1.951545
294,921,918,1,922,кубарем,роль,-1.972080


,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent,inflected_candidate,product
0,10,0,0,9,преследовал,женщину,8.436619
1,18,0,0,19,гналась,женщина,1.514307
2,30,0,0,31,оставался,женщине,3.133473
5,48,6,3,46,оторваться,сна,8.170823
9,53,0,0,52,подвели,женщину,5.742084
10,56,0,0,57,упала,женщина,12.184425
13,83,65,2,84,увидела,женщина,15.119053
18,113,95,4,114,напугала,девушка,5.132144
24,122,95,5,123,чувствовала,девушка,15.705741
32,145,105,4,146,подкараулили,кирпичного,4.122867


Стало ли лучше? Да, немного, стоит склонять. Кандидаты с расстоянием 5+ часто неправильные.
Кроме того оставлю в кандидатах только 'NOUN' и 'PROPN'.

In [3]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

alg = AntecedentCandidatesAlgorithm(reader.get_bundles(), 7)

c:\users\alexandra\desktop\grammar_ru\venv\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  torch.from_numpy(navec.pq.indexes),


In [13]:
alg.run_full(db)

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,pronoun_parent,inflected_candidate,product
0,10,0,0,9,преследовал,женщину,8.436619
1,18,0,0,19,гналась,женщина,1.514307
2,30,0,0,31,оставался,женщине,3.133473
4,48,6,2,46,оторваться,сна,8.170823
7,53,0,0,52,подвели,женщину,5.742084
8,56,0,0,57,упала,женщина,12.184425
11,83,65,1,84,увидела,женщина,15.119053
18,113,95,1,114,напугала,девушка,5.132144
24,122,95,2,123,чувствовала,девушка,15.705741
33,145,142,0,146,подкараулили,женщину,1.016804


Ок, проверка произведения для пары (кандидат, родитель местоимения) всё-таки даёт больше ошибок чем хотелось бы. Что если смотреть произведение (кандидат, сосед местоимения)?

In [6]:
candidates_df = alg.get_candidates(db)
with_right = candidates_df.copy()
with_right['right_id'] = candidates_df['pronoun_word_id'].apply(lambda x: x + 1 if db.src.loc[x + 1].word_type != 'punct' else None)
with_right = with_right.dropna().astype(int)
with_right

,pronoun_word_id,candidate_word_id,candidate_distance,right_id
1,18,0,0,19
2,30,0,0,31
8,56,0,0,57
9,83,0,3,84
10,83,61,2,84
...,...,...,...,...
288,921,896,4,922
289,921,907,3,922
290,921,914,2,922
291,921,918,1,922


In [41]:
from tg.grammar_ru.ml.features import NavecFeaturizer
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def inflect(word, case):
    return morph.parse(word)[0].inflect({'sing', case}).word

def get_inflected_candidates(db, with_parents_df):
    inflect_base = with_parents_df.merge(db.pymorphy, left_on='pronoun_word_id', right_on='word_id')[['candidate_word_id', 
                                                                                                 'case']]
    inflect_base = inflect_base.merge(db.pymorphy.reset_index()[['word_id', 'normal_form']], left_on='candidate_word_id', right_on='word_id', how='left')[['case', 'normal_form']]
    return inflect_base.apply(lambda x: inflect(x['normal_form'], x['case']), axis=1)

prod_df = with_right.merge(db.src.word.str.lower().rename('right'), left_on='right_id', right_index=True, how='left').reset_index(drop=True)
prod_df['inflected_candidate'] = get_inflected_candidates(db, with_right)
featurizer = NavecFeaturizer()
prod_df['product'] = featurizer.get_glove_prod(prod_df, 'right', 'inflected_candidate')

def get_big_deviation_only(df):
    product_groups = df.drop_duplicates(['right','inflected_candidate'], keep='last').groupby(['pronoun_word_id'])['product']
    maxes = product_groups.max()
    means = product_groups.mean()
    stds = product_groups.std()
    max_indices = product_groups.idxmax().dropna().astype(int)
    best_in_group = df.loc[max_indices]
    temp = ((product_groups.count() < 3) | (stds < maxes - means)).to_frame()
    return best_in_group[best_in_group.pronoun_word_id.isin(temp[temp['product']].index)]

get_big_deviation_only(prod_df)

,pronoun_word_id,candidate_word_id,candidate_distance,right_id,right,inflected_candidate,product
0,18,0,0,19,гналась,женщина,1.514307
1,30,0,0,31,оставался,женщине,3.133473
2,56,0,0,57,упала,женщина,12.184425
5,83,65,1,84,увидела,женщина,15.119053
12,113,95,1,114,напугала,девушка,5.132144
18,122,95,2,123,чувствовала,девушка,15.705741
27,145,142,0,146,подкараулили,женщину,1.016804
30,160,95,4,161,даже,девушка,9.747227
37,174,95,4,175,показалось,девушке,11.728028
49,224,95,6,225,мысленно,девушка,10.007082


Просто интересно попробовать: посчитать произведения для пар (кандидат, родитель), (кандидат, сосед слева), (кандидат, сосед справа)

In [4]:
from tg.grammar_ru.algorithms import AntecedentCandidatesAlgorithm

alg = AntecedentCandidatesAlgorithm(reader.get_bundles(), 7)

In [13]:
from tg.grammar_ru.ml.features import NavecFeaturizer
import numpy as np
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def inflect(word, case):
    return morph.parse(word)[0].inflect({'sing', case}).word

def get_inflected_candidates(db, with_parents_df):
    inflect_base = with_parents_df.merge(db.pymorphy, left_on='pronoun_word_id', right_on='word_id')[['candidate_word_id', 
                                                                                                 'case']]
    inflect_base = inflect_base.merge(db.pymorphy.reset_index()[['word_id', 'normal_form']], left_on='candidate_word_id', right_on='word_id', how='left')[['case', 'normal_form']]
    return inflect_base.apply(lambda x: inflect(x['normal_form'], x['case']), axis=1)

def get_glove_prod_e_norm(df, col1, col2):
    copy = df.copy()
    featurizer = NavecFeaturizer()
    copy['prod_e'] = featurizer.get_glove_prod(copy, col1, col2).apply(lambda x: np.e ** x)
    return copy.groupby('pronoun_word_id')['prod_e'].transform(lambda x: x / x.sum()).fillna(0)

def get_extended_df(db, with_parents_df):
    extended_df = with_parents_df.copy()
    extended_df['right_id'] = candidates_df['pronoun_word_id'].apply(lambda x: x + 1 if db.src.loc[x + 1].word_type != 'punct' else -1)
    extended_df['left_id'] = candidates_df['pronoun_word_id'].apply(lambda x: x - 1 if (x > 0 and db.src.loc[x - 1].word_type != 'punct') else -1)

    extended_df['inflected_candidate'] = get_inflected_candidates(db, extended_df)
    extended_df = extended_df.drop_duplicates(['inflected_candidate', 'pronoun_word_id'],keep='last').reset_index(drop=True)
    extended_df = extended_df.merge(db.src.word.str.lower().rename('right_word'), left_on='right_id', right_index=True, how='left').reset_index(drop=True)
    extended_df = extended_df.merge(db.src.word.str.lower().rename('left_word'), left_on='left_id', right_index=True, how='left').reset_index(drop=True)
    extended_df = extended_df.merge(db.src.word.str.lower().rename('parent_word'), left_on='pronoun_parent_id', right_index=True, how='left')
    
    extended_df['left_prod_e'] = get_glove_prod_e_norm(extended_df, 'inflected_candidate', 'left_word')
    extended_df['right_prod_e'] = get_glove_prod_e_norm(extended_df, 'inflected_candidate', 'right_word')
    extended_df['parent_prod_e'] = get_glove_prod_e_norm(extended_df, 'inflected_candidate', 'parent_word')
    extended_df['sum'] = extended_df[['left_prod_e', 'right_prod_e', 'parent_prod_e']].sum(axis=1)
    return extended_df

In [43]:
import pandas as pd
import numpy as np

db = reader.get_bundles().first()
candidates_df = alg.get_candidates(db)
db.pymorphy['normal_form'] = db.pymorphy['normal_form'].apply(lambda x: alg.pronoun_replacer[x] if x in alg.pronoun_replacer else x)

extended_df = get_extended_df(db, alg.get_pronoun_parent(db, candidates_df))
extended_df

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,right_id,left_id,inflected_candidate,right_word,left_word,parent_word,left_prod_e,right_prod_e,parent_prod_e,sum
0,10,0,0,9,-1,9,женщину,NaN,преследовал,преследовал,1.000000,0.000000,1.000000,2.000000
1,18,0,0,19,19,-1,женщина,гналась,NaN,гналась,0.000000,1.000000,1.000000,2.000000
2,30,0,0,31,31,-1,женщине,оставался,NaN,оставался,0.000000,1.000000,1.000000,2.000000
3,48,3,3,46,-1,47,суетливого,NaN,от,оторваться,0.001035,0.000000,0.000233,0.001267
4,48,6,2,46,-1,47,сна,NaN,от,оторваться,0.992254,0.000000,0.887464,1.879718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,921,896,4,922,922,-1,женщина,кубарем,NaN,кубарем,0.000000,0.000492,0.000492,0.000984
240,921,907,3,922,922,-1,жертва,кубарем,NaN,кубарем,0.000000,0.000228,0.000228,0.000456
241,921,914,2,922,922,-1,судьба,кубарем,NaN,кубарем,0.000000,0.000086,0.000086,0.000173
242,921,918,1,922,922,-1,роль,кубарем,NaN,кубарем,0.000000,0.000084,0.000084,0.000169


In [42]:
max_indices = extended_df.groupby('pronoun_word_id')['sum'].idxmax().dropna().astype(int)
best_in_group = extended_df.loc[max_indices]
best_in_group

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,right_id,left_id,inflected_candidate,right_word,left_word,parent_word,left_prod_e,right_prod_e,parent_prod_e,sum
0,10,0,0,9,-1,9,женщину,NaN,преследовал,преследовал,1.000000,0.000000,1.000000,2.000000
1,18,0,0,19,19,-1,женщина,гналась,NaN,гналась,0.000000,1.000000,1.000000,2.000000
2,30,0,0,31,31,-1,женщине,оставался,NaN,оставался,0.000000,1.000000,1.000000,2.000000
4,48,6,2,46,-1,47,сна,NaN,от,оторваться,0.992254,0.000000,0.887464,1.879718
7,53,0,0,52,-1,52,женщину,NaN,подвели,подвели,1.000000,0.000000,1.000000,2.000000
8,56,0,0,57,57,55,женщина,упала,и,упала,1.000000,1.000000,1.000000,3.000000
10,83,65,1,84,84,82,женщина,увидела,спальни,увидела,0.000002,0.999849,0.999849,1.999700
17,113,95,1,114,114,112,девушка,напугала,которых,напугала,0.182031,0.668873,0.668873,1.519777
23,122,95,2,123,123,121,девушка,чувствовала,и,чувствовала,0.367266,0.927165,0.927165,2.221596
32,145,142,0,146,146,-1,женщину,подкараулили,NaN,подкараулили,0.000000,0.513451,0.513451,1.026901


Если ещё уменьшить количество кандидатов:

In [46]:
alg = AntecedentCandidatesAlgorithm(reader.get_bundles(), 5)

In [57]:
db = reader.get_bundles().first()
candidates_df = alg.get_candidates(db)
db.pymorphy['normal_form'] = db.pymorphy['normal_form'].apply(lambda x: alg.pronoun_replacer[x] if x in alg.pronoun_replacer else x)
extended_df = get_extended_df(db, alg.get_pronoun_parent(db, candidates_df))

max_indices = extended_df.groupby('pronoun_word_id')['sum'].idxmax().dropna().astype(int)
best_in_group = extended_df.loc[max_indices]
best_in_group

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,right_id,left_id,inflected_candidate,right_word,left_word,parent_word,left_prod_e,right_prod_e,parent_prod_e,sum
0,10,0,0,9,-1,9,женщину,NaN,преследовал,преследовал,1.000000,0.000000,1.000000,2.000000
1,18,0,0,19,19,-1,женщина,гналась,NaN,гналась,0.000000,1.000000,1.000000,2.000000
2,30,0,0,31,31,-1,женщине,оставался,NaN,оставался,0.000000,1.000000,1.000000,2.000000
4,48,6,2,46,-1,47,сна,NaN,от,оторваться,0.992254,0.000000,0.887464,1.879718
7,53,0,0,52,-1,52,женщину,NaN,подвели,подвели,1.000000,0.000000,1.000000,2.000000
8,56,0,0,57,57,55,женщина,упала,и,упала,1.000000,1.000000,1.000000,3.000000
10,83,65,1,84,84,82,женщина,увидела,спальни,увидела,0.000002,0.999849,0.999849,1.999700
15,113,95,1,114,114,112,девушка,напугала,которых,напугала,0.335821,0.746716,0.746716,1.829253
19,122,95,2,123,123,121,девушка,чувствовала,и,чувствовала,0.516609,0.996253,0.996253,2.509114
26,145,142,0,146,146,-1,женщину,подкараулили,NaN,подкараулили,0.000000,0.534172,0.534172,1.068345


Иногда криво определяются родители, может, брать произведения с соседями x2?

In [94]:
extended_df['left_prod_e'] = extended_df['left_prod_e'] * 2
extended_df['right_prod_e'] = extended_df['right_prod_e'] * 2
extended_df['sum'] = extended_df[['left_prod_e', 'right_prod_e', 'parent_prod_e']].sum(axis=1)
max_indices = extended_df.groupby('pronoun_word_id')['sum'].idxmax().dropna().astype(int)
best_in_group = extended_df.loc[max_indices]
best_in_group

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,right_id,left_id,inflected_candidate,right_word,left_word,parent_word,left_prod_e,right_prod_e,parent_prod_e,sum
0,10,0,0,9,-1,9,женщину,NaN,преследовал,преследовал,8.000000,0.000000,1.000000,9.000000
1,18,0,0,19,19,-1,женщина,гналась,NaN,гналась,0.000000,8.000000,1.000000,9.000000
2,30,0,0,31,31,-1,женщине,оставался,NaN,оставался,0.000000,8.000000,1.000000,9.000000
4,48,6,2,46,-1,47,сна,NaN,от,оторваться,7.938034,0.000000,0.887464,8.825498
7,53,0,0,52,-1,52,женщину,NaN,подвели,подвели,8.000000,0.000000,1.000000,9.000000
8,56,0,0,57,57,55,женщина,упала,и,упала,8.000000,8.000000,1.000000,17.000000
10,83,65,1,84,84,82,женщина,увидела,спальни,увидела,0.000014,7.998792,0.999849,8.998656
15,113,95,1,114,114,112,девушка,напугала,которых,напугала,2.686569,5.973727,0.746716,9.407012
19,122,95,2,123,123,121,девушка,чувствовала,и,чувствовала,4.132870,7.970020,0.996253,13.099143
26,145,142,0,146,146,-1,женщину,подкараулили,NaN,подкараулили,0.000000,4.273380,0.534172,4.807552


In [74]:
l = extended_df[extended_df.pronoun_word_id == 174]
l

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,right_id,left_id,inflected_candidate,right_word,left_word,parent_word,left_prod_e,right_prod_e,parent_prod_e,sum
32,174,95,4,175,175,173,девушке,показалось,сначала,показалось,0.301898,2.937042,0.734260,3.973200
33,174,106,3,175,175,173,стене,показалось,сначала,показалось,0.184995,0.261641,0.065410,0.512047
34,174,118,2,175,175,173,голове,показалось,сначала,показалось,3.421241,0.473920,0.118480,4.013641
35,174,124,1,175,175,173,тревоге,показалось,сначала,показалось,0.001085,0.000029,0.000007,0.001121
36,174,142,0,175,175,173,женщине,показалось,сначала,показалось,0.090782,0.327367,0.081842,0.499991


In [131]:
def filter_max_sums(df):
    first_max_indices = df.groupby('pronoun_word_id')['sum'].idxmax()
    first_maxes = df.loc[first_max_indices][['pronoun_word_id', 'sum']]
    without_maxes = df.drop(first_max_indices.to_list())
    second_maxes = without_maxes.loc[without_maxes.groupby('pronoun_word_id')['sum'].idxmax()][['pronoun_word_id', 'sum']]
    cmp = first_maxes.merge(second_maxes, on='pronoun_word_id', how='left').fillna(0)
    cmp['diff'] = cmp['sum_x'] - cmp['sum_y']
    cmp['std'] = extended_df.groupby('pronoun_word_id')['sum'].std().fillna(0).reset_index(drop=True)
    good_diff = cmp[cmp['diff'] > cmp['std']]['pronoun_word_id'].to_list()
    return first_max_indices[good_diff].to_list()
    
extended_df.loc[filter_max_sums(extended_df)]

,pronoun_word_id,candidate_word_id,candidate_distance,pronoun_parent_id,right_id,left_id,inflected_candidate,right_word,left_word,parent_word,left_prod_e,right_prod_e,parent_prod_e,sum
0,10,0,0,9,-1,9,женщину,NaN,преследовал,преследовал,8.000000,0.000000,1.000000,9.000000
1,18,0,0,19,19,-1,женщина,гналась,NaN,гналась,0.000000,8.000000,1.000000,9.000000
2,30,0,0,31,31,-1,женщине,оставался,NaN,оставался,0.000000,8.000000,1.000000,9.000000
4,48,6,2,46,-1,47,сна,NaN,от,оторваться,7.938034,0.000000,0.887464,8.825498
7,53,0,0,52,-1,52,женщину,NaN,подвели,подвели,8.000000,0.000000,1.000000,9.000000
8,56,0,0,57,57,55,женщина,упала,и,упала,8.000000,8.000000,1.000000,17.000000
15,113,95,1,114,114,112,девушка,напугала,которых,напугала,2.686569,5.973727,0.746716,9.407012
19,122,95,2,123,123,121,девушка,чувствовала,и,чувствовала,4.132870,7.970020,0.996253,13.099143
27,160,95,4,163,161,-1,девушка,даже,NaN,успела,0.000000,4.922525,0.870104,5.792629
37,187,95,4,188,188,186,девушку,глазах,в,глазах,2.973235,3.470133,0.433767,6.877135


In [77]:
l['sum'].std()

2.0129466243880083

In [78]:
4.01 - l['sum'].mean()

2.21